In [ ]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as functional
from torchvision import datasets
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
batch_size = 64

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size=(5,9), sigma=(0.1, 5)),
    transforms.RandomRotation(degrees=(30, 70)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

In [ ]:
train_dataset = datasets.ImageFolder(
    root='/kaggle/input/rating-opencv-emotion-images/Images/train',
    transform=train_transform
)

validation_dataset = datasets.ImageFolder(
    root='/kaggle/input/rating-opencv-emotion-images/Images/validation',
    transform=valid_transform
)

In [ ]:
train_loader = DataLoader(
 train_dataset, batch_size=batch_size, shuffle=True, 
 num_workers=4, pin_memory=True
)

valid_loader = DataLoader(
    validation_dataset, batch_size=batch_size, shuffle=False,
    num_workers=4, pin_memory=True
)

In [ ]:
train_dataset

In [ ]:
class ImageModel(nn.Module):
    def __init__(self):
        super(ImageModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.conv4 = nn.Conv2d(128, 256, 5)
        
        self.fc1 = nn.Linear(256, 7)
        self.pool = nn.MaxPool2d(2, 2)
        
    def forward(self, x):
        x = self.pool(functional.relu(self.conv1(x)))
        x = self.pool(functional.relu(self.conv2(x)))
        x = self.pool(functional.relu(self.conv3(x)))
        x = self.pool(functional.relu(self.conv4(x)))
        
        bs, _, _, _ = x.shape
        x = functional.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        x = self.fc1(x)
        return x
    


In [ ]:
lr = 1e-3
epochs = 30

device = ('cuda' if torch.cuda.is_available() else 'cpu')
image_classifier = ImageModel().to(device)
optimizer = optim.Adam(image_classifier.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

device, image_classifier

In [ ]:
total_params = sum(p.numel() for p in image_classifier.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in image_classifier.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

In [ ]:
def training(model, trainloader, optimizer, criterion):
    model.train()
    print('Training torch classifier')
    training_run_loss = 0.0
    training_run_correct = 0
    counter = 0
    for x, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        counter += 1
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(image)
        loss = criterion(outputs, labels)
        training_run_loss += loss.item()
        _, preds = torch.max(outputs.data, 1)
        training_run_correct += (preds == labels).sum().item()
        
        loss.backward()
        optimizer.step()
        
    epoch_loss = training_run_loss / counter
    epoch_acc = 100. * (training_run_correct / len(train_loader.dataset))
    
def validate(model, valid_loader, criterion):
    model.eval()
    print('Validation')
    valid_run_loss = 0.0
    valid_run_correct = 0
    counter = 0
    
    with torch.no_grad():
        for i, data in tqdm(enumerate(valid_loader), total=len(valid_loader)):
            counter += 1
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            
            outputs = model(image)
            
            loss = criterion(outputs, labels)
            valid_run_loss += loss.item()
            _, preds = torch.max(outputs.data, 1)
            valid_run_correct += (preds == labels).sum().item()
            
    epoch_loss = valid_run_loss / counter
    epoch_acc = 100. * (valid_run_correct / len(valid_loader.dataset))
    return epoch_loss, epoch_acc


In [ ]:
def save_plots(train_acc, valid_acc, train_loss, valid_loss):
    """
    Function to save the loss and accuracy plots to disk.
    """
    # accuracy plots
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_acc, color='green', linestyle='-', 
        label='train accuracy'
    )
    plt.plot(
        valid_acc, color='blue', linestyle='-', 
        label='validataion accuracy'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig('outputs/accuracy.png')
    
    # loss plots
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_loss, color='orange', linestyle='-', 
        label='train loss'
    )
    plt.plot(
        valid_loss, color='red', linestyle='-', 
        label='validation loss'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('outputs/loss.png')

In [ ]:
train_loss, valid_loss = [], []
train_acc, valid_acc = [], []

for epoch in tqdm(range(epochs)):
    print(f'[INFO]: Epoch {epoch+1} of {epochs}')
    train_epoch_loss, train_epoch_acc = training(image_classifier, train_loader, optimizer, criterion)
    valid_epoch_loss, valid_epoch_acc = validate(image_classifier, valid_loader, criterion)
    
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    
    print(f"Training loss: {train_epoch_loss:.3f}, training acc: {train_epoch_acc:.3f * 100}")
    print(f"Validation loss: {valid_epoch_loss:.3f}, validation acc: {valid_epoch_acc:.3f * 100}")
    print('-'*50)
    time.sleep(5)
    
save_model(epochs, image_classifier, optimizer, criterion)
save_plots(train_acc, valid_acc, train_loss, valid_loss)
print('Training Complete')